In [1]:
!pip install "dask[complete]"

In [2]:
!pip install mlxtend

**Import the necessary Libraries**



In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import os
import dask.dataframe as dd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pd.set_option('display.max_colwidth', None)

**Read and Preprocess Data**



In [4]:

# Define the data directory
data_dir = Path('/content/drive/MyDrive/Data_hasking_without_banker_account')

# Load all Parquet files in the directory
data = dd.read_parquet(str(data_dir / '*.parquet'))


In [5]:
data_pd=data.compute()

In [7]:

# Display the first few rows of the dataframe
data_pd.head()


,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
709,af3e6c0fbc6207736f6c97671d63e75cfc109129d673881416e12fa47f8cf5d8,CASA,3fdb8894344ab1289cec0efdd03ef870cfc23459f2e961fc1b3158d54f078b94,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011219361,VND,414.5,414.5,110000,...,XX.XX.XXXX sXXXXXXXX bcXXXX,N,21|45,c016c5efb5b3fe72156e1fed3593384def261de1811b3f5af1225633a81eeeb1,STP,HOI SO SHB,LP,CITAD SHB,NaN,20230801
894,c8ea05c1c35e2ed34d5b1d3c32950cc8fb99074b92d251a1a8788dab9750c40a,CASA,526978799ca402ab0f99c4c565917976bd22ee37dfb6f20885cf12773b260bc9,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011174207,VND,-510.0,-510.0,110000,...,<NA>,N,1|1,b19d1ca799a45a0483e0a49d0eb7504ad6d8012f8bf23374c34b166dc5ead6e0,MOB,HOI SO SHB,A2,TRAN CHI AN,NaN,20230801
902,2bdb75e6139a7717939d04a1ef7b8a7e17fe3261064a309c02aac70c7a447709,CASA,c60feae21e6d3a79d1b9dca82edf74b97b9e0bd93c461c51b092b7bdbfe8777d,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011205699,VND,500.0,500.0,110000,...,<NA>,N,1|2,8ea1260216b3cb219d1f797e1b678a71563151d09695fc3ae74db637b9933a02,MOB,HOI SO SHB,A2,VU THU HIEN,NaN,20230801
904,c8ea05c1c35e2ed34d5b1d3c32950cc8fb99074b92d251a1a8788dab9750c40a,CASA,526978799ca402ab0f99c4c565917976bd22ee37dfb6f20885cf12773b260bc9,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011192048,VND,1000.0,1000.0,110000,...,<NA>,N,1|2,adb8fa7e91207bdb4066688c0691a483fc7a60cb3e065d5f3f9361500a1bc727,MOB,HOI SO SHB,A2,NGUYEN TIEN SY,NaN,20230801
907,2bdb75e6139a7717939d04a1ef7b8a7e17fe3261064a309c02aac70c7a447709,CASA,c60feae21e6d3a79d1b9dca82edf74b97b9e0bd93c461c51b092b7bdbfe8777d,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011167253,VND,-500.0,-500.0,110000,...,<NA>,N,1|1,a390b60d34d64407a224af176bb5f2fd44994e48b7a6e469edbf468b2dab4e63,MOB,HOI SO SHB,A2,VU THU HIEN,NaN,20230801


In [9]:
data.columns

Index(['CUST_CUSTNO', 'ACC_BUSINESSTYPE', 'ACC_ACCNO', 'ACC_BUSINESSNO',
       'ACC_CURRENCYISO', 'BUSINESSNO_TRANS', 'FK_CURRENCY', 'AMOUNT',
       'AMOUNTORIG', 'BRANCH_OFFICE', 'CONTRA_ACCNO', 'CONTRA_ZIP',
       'CONTRA_NAME', 'CSHYN', 'REASON1', 'REASON2', 'REASON4', 'TR_FLAG_01',
       'ANALYTICAL_TRANS_CODE', 'TR_SP_01', 'TR_SP_02', 'TR_SP_03', 'TR_SP_04',
       'TR_SP_05', 'TR_SP_10', 'VALUEDATE'],
      dtype='object')

In [10]:
data.dtypes

CUST_CUSTNO              string[pyarrow]
ACC_BUSINESSTYPE         string[pyarrow]
ACC_ACCNO                string[pyarrow]
ACC_BUSINESSNO           string[pyarrow]
ACC_CURRENCYISO          string[pyarrow]
BUSINESSNO_TRANS         string[pyarrow]
FK_CURRENCY              string[pyarrow]
AMOUNT                           float64
AMOUNTORIG                       float64
BRANCH_OFFICE            string[pyarrow]
CONTRA_ACCNO             string[pyarrow]
CONTRA_ZIP               string[pyarrow]
CONTRA_NAME              string[pyarrow]
CSHYN                    string[pyarrow]
REASON1                  string[pyarrow]
REASON2                  string[pyarrow]
REASON4                  string[pyarrow]
TR_FLAG_01               string[pyarrow]
ANALYTICAL_TRANS_CODE    string[pyarrow]
TR_SP_01                 string[pyarrow]
TR_SP_02                 string[pyarrow]
TR_SP_03                 string[pyarrow]
TR_SP_04                 string[pyarrow]
TR_SP_05                 string[pyarrow]
TR_SP_10        

In [6]:
# Convert the VALUEDATE column to datetime format
data_pd['VALUEDATE'] = dd.to_datetime(data_pd['VALUEDATE'])

# Define the two specific dates you are interested in
date1 = '2024-01-01'
date2 = '2024-01-02'

# Filter the data for the two specific dates
filtered_data = data_pd[(data_pd['VALUEDATE'] == date1) | (data_pd['VALUEDATE'] == date2)]


In [12]:
filtered_data

,CUST_CUSTNO,ACC_BUSINESSTYPE,ACC_ACCNO,ACC_BUSINESSNO,ACC_CURRENCYISO,BUSINESSNO_TRANS,FK_CURRENCY,AMOUNT,AMOUNTORIG,BRANCH_OFFICE,...,REASON4,TR_FLAG_01,ANALYTICAL_TRANS_CODE,TR_SP_01,TR_SP_02,TR_SP_03,TR_SP_04,TR_SP_05,TR_SP_10,VALUEDATE
3203,16d2f73347cc573572e6bdd6697d0180581a3aa9d3212ac54cc70751234189a5,CASA,592a826aa75c561decf2673dc9a707e160b0e32d94457b0e35432ac0c22e4f00,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011206733,VND,520.0,520.0,110000,...,<NA>,N,1|2,d3ead670b52fc2f03f7bc80a4ccd3280f9fe29ac266035f9fdb353035090ebf1,MOB,HOI SO SHB,A2,PHAN THI HIEU,NaN,2024-01-01
4025,78e262b74febdcb384a8db90f451cd5077fc02fb20eda30e768dcb8683b77cdf,CASA,40448666689d66046d93993f4986521210065db70409a111eb7f85ddddb4c9e1,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011202480,VND,200.0,200.0,110000,...,<NA>,N,1|2,946063a33ffe65d254dba56a4b1446a1e71e601cc5e2f155db44ea0ceefd1c44,MOB,HOI SO SHB,A2,LO VAN DOAN,NaN,2024-01-01
5805,5ef0eaa1a2143bde7c1b818f112cdc19c335124b0314f57bb1aead72389b07ae,CASA,f87c55cb5705ee7bf25a27cdcd25fd5e3903dddc4588ac0e049ddc13c191e565,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011187503,VND,-400.0,-400.0,110000,...,<NA>,Y,1|1,25bae560099e50a9b6f59afc7fac3998a950a1181bcdcb2944a723f77d826487,NET,HOI SO SHB,A2,PHAM VAN TRUC,NaN,2024-01-01
5831,5ef0eaa1a2143bde7c1b818f112cdc19c335124b0314f57bb1aead72389b07ae,CASA,f87c55cb5705ee7bf25a27cdcd25fd5e3903dddc4588ac0e049ddc13c191e565,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011187502,VND,-500.0,-500.0,110000,...,<NA>,Y,1|1,96cefafd3215eba5946bdff0a36c22fd2fec4681bfc1506f65461d5a4b560ded,NET,HOI SO SHB,A2,LE HONG QUAN,NaN,2024-01-01
5835,5ef0eaa1a2143bde7c1b818f112cdc19c335124b0314f57bb1aead72389b07ae,CASA,f87c55cb5705ee7bf25a27cdcd25fd5e3903dddc4588ac0e049ddc13c191e565,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,202308011187507,VND,-500.0,-500.0,110000,...,<NA>,Y,1|1,d164b245e45f6dd18464fe090f1e7711182fab2d872a470fe81c0abc01629df2,NET,HOI SO SHB,A2,NGUYEN HAI BANG,NaN,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231359,d2e0d5e8436ef851c6281e4350091b385037f290ed3b56540cccaff8c9b52ef0,CASA,4fa753e170d44ce21c23674de4e97551e9916581703e05904361a5f357202a7c,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024012940149439,VND,35.0,35.0,110000,...,<NA>,N,1|2,2b9f8303e2c5fbf03ca374fcaa9ecd91e012197011e4b3d84a1fd155b070b2ca,MOB,HOI SO SHB,A2,DAO QUANG HUY,NaN,2024-01-02
231372,cde577286da34134144cf85e41fac765007a5fbc18e9a56404e09ab7b1d61a33,CASA,2adcf9301d120fa1e027b1fbfd35f58bff9064340306114b7a3e0a3e994019bd,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024012940149522,VND,33.0,33.0,110000,...,<NA>,N,1|2,3c77cd4a52b6387936019627a055b6fa78bd56f53726a342ed94eb02770dc04a,MOB,HOI SO SHB,A2,PHAM ANH TUAN,NaN,2024-01-02
231443,742617df388f184bda0e0ab83f7839df8fd1343712ef2637ad9ec8cd49289a8d,CASA,06287fae3d5db0675e249e50917b6c7b0b393c979dd5ef8e31de139b604de0ed,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024012940097774,VND,-150.0,-150.0,110000,...,<NA>,N,1|1,5210fa5cc2eec2be5a3d2f903751b770179108bd72bdfcad9f530676ab957618,MOB,HOI SO SHB,A2,PHAN TUNG DUONG,NaN,2024-01-02
231720,2181209b470ac72eab26f0a541cd493947828c0d251c6ceba50f7865a89a44da,CASA,0a9f396e746e9a64326324d307c50b42c6e4c9ced846b5efa600505e3c945665,0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6,VND,2024012940149220,VND,31000.0,31000.0,110000,...,<NA>,N,1|2,250413370a41d9fb22545e089fd341e22f3f85137c7742f11fc0b6c278e1587e,MOB,HOI SO SHB,A2,NGUYEN HUU HAI,NaN,2024-01-02


In [7]:
# Check for missing values
missing_values = filtered_data.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Drop columns with a high percentage of missing values (e.g., more than 50%)
threshold = len(filtered_data) * 0.5
filtered_data = filtered_data.dropna(thresh=threshold, axis=1)

# For the remaining columns, fill missing values with appropriate strategies
# For numeric columns, fill with the median
numeric_cols = filtered_data.select_dtypes(include=['number']).columns
filtered_data[numeric_cols] = filtered_data[numeric_cols].fillna(filtered_data[numeric_cols].median())

# For string columns, fill with the mode
string_cols = filtered_data.select_dtypes(include=['string']).columns
filtered_data[string_cols] = filtered_data[string_cols].apply(lambda x: x.fillna(x.mode()[0]))

# Verify that there are no missing values left
print("Missing values after cleaning:")
print(filtered_data.isnull().sum())


Missing values per column:
CUST_CUSTNO                   0
ACC_BUSINESSTYPE            671
ACC_ACCNO                     0
ACC_BUSINESSNO                0
ACC_CURRENCYISO             671
BUSINESSNO_TRANS              0
FK_CURRENCY                   0
AMOUNT                        0
AMOUNTORIG                    0
BRANCH_OFFICE                 0
CONTRA_ACCNO                  0
CONTRA_ZIP                   55
CONTRA_NAME                   0
CSHYN                         0
REASON1                      12
REASON2                   20913
REASON4                   86774
TR_FLAG_01                    0
ANALYTICAL_TRANS_CODE         0
TR_SP_01                      0
TR_SP_02                      0
TR_SP_03                      0
TR_SP_04                      0
TR_SP_05                      0
TR_SP_10                 103391
VALUEDATE                     0
dtype: int64
Missing values after cleaning:
CUST_CUSTNO              0
ACC_BUSINESSTYPE         0
ACC_ACCNO                0
ACC_BUSINESSNO  

In [14]:
# Convert the dataset to a list of lists
#transactions = filtered_data.values.tolist()

In [15]:
#transactions

[['16d2f73347cc573572e6bdd6697d0180581a3aa9d3212ac54cc70751234189a5',
  'CASA',
  '592a826aa75c561decf2673dc9a707e160b0e32d94457b0e35432ac0c22e4f00',
  '0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6',
  'VND',
  '202308011206733',
  'VND',
  520.0,
  520.0,
  '110000',
  '1b23dce68e0e8bedcc9e10e5b1742d0c458ceadfa590f8c174b614bca8e25e93',
  'SHBAVNVX',
  'PHAN THI HIEU',
  'N',
  'PHAN THI HIEU  CHUYEN KHOAN',
  ' NGUYEN HUU THANH',
  'N',
  '1|2',
  'd3ead670b52fc2f03f7bc80a4ccd3280f9fe29ac266035f9fdb353035090ebf1',
  'MOB',
  'HOI SO SHB',
  'A2',
  'PHAN THI HIEU',
  Timestamp('2024-01-01 00:00:00')],
 ['78e262b74febdcb384a8db90f451cd5077fc02fb20eda30e768dcb8683b77cdf',
  'CASA',
  '40448666689d66046d93993f4986521210065db70409a111eb7f85ddddb4c9e1',
  '0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6',
  'VND',
  '202308011202480',
  'VND',
  200.0,
  200.0,
  '110000',
  '342e87dc1e444440a8af20a1833ca247a190e86183d3c73adb6f6dcb41b7c34f',
  'SHBAVNVX

In [15]:

# Set high values for min_support and min_threshold
min_support = 0.1  # Adjust as per your requirement
min_confidence = 2  # Adjust as per your requirement



In [17]:

# Apply the Apriori algorithm with high min_support
#frequent_itemsets = apriori(transactions, min_support=min_support, use_colnames=True)

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 16d2f73347cc573572e6bdd6697d0180581a3aa9d3212ac54cc70751234189a5

the Apriori algorithm from the mlxtend library is expecting binary (0/1) or boolean values in the DataFrame, but it encountered non-binary or non-boolean values in one or more columns.

In [8]:
# Convert categorical columns into binary format using one-hot encoding
one_hot_encoded_data = pd.get_dummies(filtered_data)


In [9]:

# Convert the DataFrame to a transaction DataFrame
transaction_df = one_hot_encoded_data.astype(bool)

In [13]:
#  make the apriori algorithm

frequent_itemsets = apriori(transaction_df, min_support=min_support, use_colnames=True)


In [14]:
frequent_itemsets

,support,itemsets
0,0.991353,(AMOUNT)
1,0.991353,(AMOUNTORIG)
2,1.000000,(VALUEDATE)
3,1.000000,(ACC_BUSINESSTYPE_CASA)
4,1.000000,(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6)
...,...,...
67834,0.396456,"(TR_SP_02_MOB, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, ANALYTICAL_TRANS_CODE_1|1, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, TR_SP_04_A2, ACC_CURRENCYISO_VND)"
67835,0.395595,"(TR_SP_02_MOB, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, TR_SP_04_A2, ANALYTICAL_TRANS_CODE_1|2, ACC_CURRENCYISO_VND)"
67836,0.174551,"(TR_SP_02_MOB, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, REASON2_N, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, AMOUNT, TR_SP_04_A2, ACC_CURRENCYISO_VND)"
67837,0.396456,"(TR_SP_02_MOB, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, ANALYTICAL_TRANS_CODE_1|1, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, AMOUNT, TR_SP_04_A2, ACC_CURRENCYISO_VND)"


In [18]:
rules# min_confidence=2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


In [22]:
# Set initial minimum confidence threshold
min_confidence = 0.1  # Start with a low value

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

# Evaluate the number and quality of generated rules
print("Number of rules with min confidence", min_confidence, ":", len(rules))
print("Example rules:")
print(rules.head())

# You can also iterate over different confidence thresholds to find the best value
# Example:
for min_confidence in [0.1, 0.2, 0.3, 0.4, 0.5]:
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    print("Number of rules with min confidence", min_confidence, ":", len(rules))


Number of rules with min confidence 0.1 : 33725642
Example rules:
               antecedents   consequents  antecedent support  \
0             (AMOUNTORIG)      (AMOUNT)            0.991353   
1                 (AMOUNT)  (AMOUNTORIG)            0.991353   
2              (VALUEDATE)      (AMOUNT)            1.000000   
3                 (AMOUNT)   (VALUEDATE)            0.991353   
4  (ACC_BUSINESSTYPE_CASA)      (AMOUNT)            1.000000   

   consequent support   support  confidence      lift  leverage  conviction  \
0            0.991353  0.991353    1.000000  1.008722  0.008572         inf   
1            0.991353  0.991353    1.000000  1.008722  0.008572         inf   
2            0.991353  0.991353    0.991353  1.000000  0.000000         1.0   
3            1.000000  0.991353    1.000000  1.000000  0.000000         inf   
4            0.991353  0.991353    0.991353  1.000000  0.000000         1.0   

   zhangs_metric  
0            1.0  
1            1.0  
2            0.0 

In [23]:
# Set initial minimum confidence threshold
min_confidence1 = 1

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence1)


In [28]:

# Evaluate the number and quality of generated rules
print("Number of rules with min confidence", min_confidence1, ":", len(rules))
print("Example rules:")
rules

Number of rules with min confidence 1 : 5061154
Example rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AMOUNTORIG),(AMOUNT),0.991353,0.991353,0.991353,1.0,1.008722,0.008572,inf,1.000000
1,(AMOUNT),(AMOUNTORIG),0.991353,0.991353,0.991353,1.0,1.008722,0.008572,inf,1.000000
2,(AMOUNT),(VALUEDATE),0.991353,1.000000,0.991353,1.0,1.000000,0.000000,inf,0.000000
3,(AMOUNT),(ACC_BUSINESSTYPE_CASA),0.991353,1.000000,0.991353,1.0,1.000000,0.000000,inf,0.000000
4,(AMOUNT),(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6),0.991353,1.000000,0.991353,1.0,1.000000,0.000000,inf,0.000000
...,...,...,...,...,...,...,...,...,...,...
5061149,"(BRANCH_OFFICE_110000, AMOUNTORIG, ANALYTICAL_TRANS_CODE_1|2, TR_FLAG_01_N)","(TR_SP_02_MOB, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, TR_SP_03_HOI SO SHB, ACC_BUSINESSTYPE_CASA, AMOUNT, TR_SP_04_A2, ACC_CURRENCYISO_VND)",0.395595,0.792052,0.395595,1.0,1.262544,0.082263,inf,0.344055
5061150,"(TR_SP_03_HOI SO SHB, ANALYTICAL_TRANS_CODE_1|2, TR_FLAG_01_N, AMOUNT)","(TR_SP_02_MOB, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, TR_SP_04_A2, ACC_CURRENCYISO_VND)",0.395595,0.792052,0.395595,1.0,1.262544,0.082263,inf,0.344055
5061151,"(BRANCH_OFFICE_110000, ANALYTICAL_TRANS_CODE_1|2, TR_FLAG_01_N, AMOUNT)","(TR_SP_02_MOB, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, TR_SP_03_HOI SO SHB, ACC_BUSINESSTYPE_CASA, TR_SP_04_A2, ACC_CURRENCYISO_VND)",0.395595,0.792052,0.395595,1.0,1.262544,0.082263,inf,0.344055
5061152,"(TR_SP_02_MOB, AMOUNTORIG, ANALYTICAL_TRANS_CODE_1|2)","(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, CONTRA_ZIP_SHBAVNVX, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, AMOUNT, TR_SP_04_A2, ACC_CURRENCYISO_VND)",0.395595,0.792322,0.395595,1.0,1.262113,0.082156,inf,0.343607


In [32]:
#  make the apriori algorithm
min_support1=1
frequent_itemsets1 = apriori(transaction_df, min_support=min_support1, use_colnames=True)

In [42]:
frequent_itemsets1

,support,itemsets
0,1.0,(VALUEDATE)
1,1.0,(ACC_BUSINESSTYPE_CASA)
2,1.0,(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6)
3,1.0,"(VALUEDATE, ACC_BUSINESSTYPE_CASA)"
4,1.0,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, VALUEDATE)"
5,1.0,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, ACC_BUSINESSTYPE_CASA)"
6,1.0,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, VALUEDATE, ACC_BUSINESSTYPE_CASA)"


In [43]:
# Set initial minimum confidence threshold
min_confidence1 = 1

# Generate association rules
rules = association_rules(frequent_itemsets1, metric="confidence", min_threshold=min_confidence1)


In [44]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(VALUEDATE),(ACC_BUSINESSTYPE_CASA),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
1,(ACC_BUSINESSTYPE_CASA),(VALUEDATE),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
2,(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6),(VALUEDATE),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
3,(VALUEDATE),(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
4,(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6),(ACC_BUSINESSTYPE_CASA),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
5,(ACC_BUSINESSTYPE_CASA),(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
6,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, VALUEDATE)",(ACC_BUSINESSTYPE_CASA),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
7,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, ACC_BUSINESSTYPE_CASA)",(VALUEDATE),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
8,"(VALUEDATE, ACC_BUSINESSTYPE_CASA)",(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6),1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0
9,(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6),"(VALUEDATE, ACC_BUSINESSTYPE_CASA)",1.0,1.0,1.0,1.0,1.0,0.0,inf,0.0


In [45]:


antecedents = list(rules['antecedents'])
consequents = list(rules['consequents'])


In [47]:
antecedents,consequents

([frozenset({'VALUEDATE'}),
  frozenset({'ACC_BUSINESSTYPE_CASA'}),
  frozenset({'ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6'}),
  frozenset({'VALUEDATE'}),
  frozenset({'ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6'}),
  frozenset({'ACC_BUSINESSTYPE_CASA'}),
  frozenset({'ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6',
             'VALUEDATE'}),
  frozenset({'ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6',
             'ACC_BUSINESSTYPE_CASA'}),
  frozenset({'ACC_BUSINESSTYPE_CASA', 'VALUEDATE'}),
  frozenset({'ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6'}),
  frozenset({'VALUEDATE'}),
  frozenset({'ACC_BUSINESSTYPE_CASA'})],
 [frozenset({'ACC_BUSINESSTYPE_CASA'}),
  frozenset({'VALUEDATE'}),
  frozenset({'VALUEDATE'}),
  frozenset({'ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee

In [51]:
#  make the apriori algorithm
min_support2=0.9
frequent_itemsets2 = apriori(transaction_df, min_support=min_support2, use_colnames=True)

In [52]:
frequent_itemsets2

,support,itemsets
0,0.991353,(AMOUNT)
1,0.991353,(AMOUNTORIG)
2,1.000000,(VALUEDATE)
3,1.000000,(ACC_BUSINESSTYPE_CASA)
4,1.000000,(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6)
...,...,...
2890,0.904760,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, AMOUNT, ACC_CURRENCYISO_VND)"
2891,0.904760,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CSHYN_N, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, AMOUNT, ACC_CURRENCYISO_VND)"
2892,0.904760,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, AMOUNT, ACC_CURRENCYISO_VND)"
2893,0.904760,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)"


In [53]:
# Set initial minimum confidence threshold
min_confidence2 =0.9

# Generate association rules
rules = association_rules(frequent_itemsets2, metric="confidence", min_threshold=min_confidence2)


In [54]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AMOUNTORIG),(AMOUNT),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.0
1,(AMOUNT),(AMOUNTORIG),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.0
2,(VALUEDATE),(AMOUNT),1.000000,0.991353,0.991353,0.991353,1.000000,0.000000,1.000000,0.0
3,(AMOUNT),(VALUEDATE),0.991353,1.000000,0.991353,1.000000,1.000000,0.000000,inf,0.0
4,(ACC_BUSINESSTYPE_CASA),(AMOUNT),1.000000,0.991353,0.991353,0.991353,1.000000,0.000000,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
232749,(TR_SP_03_HOI SO SHB),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, TR_FLAG_01_N, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, AMOUNT, ACC_CURRENCYISO_VND)",0.938012,0.904760,0.904760,0.964550,1.066084,0.056084,2.686621,1.0
232750,(BRANCH_OFFICE_110000),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, ACC_BUSINESSTYPE_CASA, AMOUNT, ACC_CURRENCYISO_VND)",0.938012,0.904760,0.904760,0.964550,1.066084,0.056084,2.686621,1.0
232751,(ACC_BUSINESSTYPE_CASA),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, AMOUNT, ACC_CURRENCYISO_VND)",1.000000,0.904760,0.904760,0.904760,1.000000,0.000000,1.000000,0.0
232752,(AMOUNT),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, TR_FLAG_01_N, TR_SP_03_HOI SO SHB, BRANCH_OFFICE_110000, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)",0.991353,0.904760,0.904760,0.912651,1.008722,0.007823,1.090345,1.0


In [55]:
#  make the apriori algorithm
min_support3=0.99
frequent_itemsets3 = apriori(transaction_df, min_support=min_support3, use_colnames=True)

In [56]:
frequent_itemsets3

,support,itemsets
0,0.991353,(AMOUNT)
1,0.991353,(AMOUNTORIG)
2,1.000000,(VALUEDATE)
3,1.000000,(ACC_BUSINESSTYPE_CASA)
4,1.000000,(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6)
...,...,...
106,0.997872,"(FK_CURRENCY_VND, CSHYN_N, VALUEDATE, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)"
107,0.997872,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, ACC_CURRENCYISO_VND)"
108,0.997872,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, CSHYN_N, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)"
109,0.991111,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, CSHYN_N, VALUEDATE, ACC_BUSINESSTYPE_CASA, AMOUNT)"


In [57]:
# Set initial minimum confidence threshold
min_confidence3 =0.99

# Generate association rules
rules = association_rules(frequent_itemsets3, metric="confidence", min_threshold=min_confidence3)


In [58]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AMOUNTORIG),(AMOUNT),0.991353,0.991353,0.991353,1.000000,1.008722,8.572021e-03,inf,1.000000
1,(AMOUNT),(AMOUNTORIG),0.991353,0.991353,0.991353,1.000000,1.008722,8.572021e-03,inf,1.000000
2,(VALUEDATE),(AMOUNT),1.000000,0.991353,0.991353,0.991353,1.000000,0.000000e+00,1.000000,0.000000
3,(AMOUNT),(VALUEDATE),0.991353,1.000000,0.991353,1.000000,1.000000,0.000000e+00,inf,0.000000
4,(ACC_BUSINESSTYPE_CASA),(AMOUNT),1.000000,0.991353,0.991353,0.991353,1.000000,0.000000e+00,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1149,(FK_CURRENCY_VND),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, CSHYN_N, VALUEDATE, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)",0.998114,0.997872,0.997872,0.999758,1.001890,1.882031e-03,8.783403,1.000000
1150,(CSHYN_N),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, VALUEDATE, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)",0.999758,0.998114,0.997872,0.998113,1.000000,-4.560465e-07,0.999758,-0.001887
1151,(VALUEDATE),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, CSHYN_N, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)",1.000000,0.997872,0.997872,0.997872,1.000000,0.000000e+00,1.000000,0.000000
1152,(ACC_BUSINESSTYPE_CASA),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, CSHYN_N, VALUEDATE, ACC_CURRENCYISO_VND)",1.000000,0.997872,0.997872,0.997872,1.000000,0.000000e+00,1.000000,0.000000


In [59]:
# prompt: extract the rules that has the zhanc metric >0

rules[rules['zhangs_metric'] > 0]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AMOUNTORIG),(AMOUNT),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.000000
1,(AMOUNT),(AMOUNTORIG),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.000000
42,(FK_CURRENCY_VND),(ACC_CURRENCYISO_VND),0.998114,0.998114,0.998114,1.000000,1.001890,0.001882,inf,1.000000
43,(ACC_CURRENCYISO_VND),(FK_CURRENCY_VND),0.998114,0.998114,0.998114,1.000000,1.001890,0.001882,inf,1.000000
48,"(AMOUNTORIG, VALUEDATE)",(AMOUNT),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.000000
...,...,...,...,...,...,...,...,...,...,...
1144,"(CSHYN_N, ACC_CURRENCYISO_VND)","(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, VALUEDATE, FK_CURRENCY_VND, ACC_BUSINESSTYPE_CASA)",0.997872,0.998114,0.997872,1.000000,1.001890,0.001882,inf,0.886364
1146,"(VALUEDATE, ACC_CURRENCYISO_VND)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, FK_CURRENCY_VND, ACC_BUSINESSTYPE_CASA)",0.998114,0.997872,0.997872,0.999758,1.001890,0.001882,8.783403,1.000000
1147,"(ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, VALUEDATE, FK_CURRENCY_VND)",0.998114,0.997872,0.997872,0.999758,1.001890,0.001882,8.783403,1.000000
1149,(FK_CURRENCY_VND),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, CSHYN_N, VALUEDATE, ACC_BUSINESSTYPE_CASA, ACC_CURRENCYISO_VND)",0.998114,0.997872,0.997872,0.999758,1.001890,0.001882,8.783403,1.000000


In [61]:
#  extract the rules that has the best value of lift, zhangs metrics and liverage

best_rules = rules.sort_values(by=['lift', 'zhangs_metric', 'leverage'], ascending=False)



In [62]:
best_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
70,(AMOUNTORIG),"(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.000000
71,(AMOUNT),"(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.000000
299,"(AMOUNTORIG, VALUEDATE)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.000000
301,"(VALUEDATE, AMOUNT)","(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.000000
303,(AMOUNTORIG),"(CSHYN_N, VALUEDATE, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.000000
...,...,...,...,...,...,...,...,...,...,...
1067,"(CSHYN_N, VALUEDATE, ACC_BUSINESSTYPE_CASA)","(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, AMOUNT)",0.999758,0.991353,0.991111,0.991351,0.999998,-0.000002,0.999758,-0.008649
1072,"(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6)","(AMOUNTORIG, VALUEDATE, ACC_BUSINESSTYPE_CASA, AMOUNT)",0.999758,0.991353,0.991111,0.991351,0.999998,-0.000002,0.999758,-0.008649
1080,"(CSHYN_N, VALUEDATE)","(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, ACC_BUSINESSTYPE_CASA, AMOUNT)",0.999758,0.991353,0.991111,0.991351,0.999998,-0.000002,0.999758,-0.008649
1081,"(CSHYN_N, ACC_BUSINESSTYPE_CASA)","(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, VALUEDATE, AMOUNT)",0.999758,0.991353,0.991111,0.991351,0.999998,-0.000002,0.999758,-0.008649


In [73]:
#  extract the rules that  only has zhangs_metric=1

rules[rules['zhangs_metric'] > 1 ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(AMOUNTORIG),(AMOUNT),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.0
1,(AMOUNT),(AMOUNTORIG),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.0
48,"(AMOUNTORIG, VALUEDATE)",(AMOUNT),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.0
50,"(VALUEDATE, AMOUNT)",(AMOUNTORIG),0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.0
51,(AMOUNTORIG),"(VALUEDATE, AMOUNT)",0.991353,0.991353,0.991353,1.000000,1.008722,0.008572,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
1078,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, VALUEDATE, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.0
1084,"(VALUEDATE, AMOUNT)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, ACC_BUSINESSTYPE_CASA)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.0
1085,"(ACC_BUSINESSTYPE_CASA, AMOUNT)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, VALUEDATE)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.0
1087,(AMOUNTORIG),"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, CSHYN_N, VALUEDATE, ACC_BUSINESSTYPE_CASA, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.008570,36.442144,1.0


**best rules Filtred by zhangs_metrics,best value of lift,support,confidence and conviction**

In [84]:
 #  extract the rules that  only has zhangs_metric>1 and conviction not inf

import numpy as np
filtered_rules = rules[(rules['zhangs_metric'] > 1) & (~rules['conviction'].isin([np.inf, -np.inf]))]

filtered_rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
70,(AMOUNTORIG),"(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
71,(AMOUNT),"(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
299,"(AMOUNTORIG, VALUEDATE)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
301,"(VALUEDATE, AMOUNT)","(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
303,(AMOUNTORIG),"(CSHYN_N, VALUEDATE, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
305,(AMOUNT),"(CSHYN_N, AMOUNTORIG, VALUEDATE)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
327,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
329,"(ACC_BUSINESSTYPE_CASA, AMOUNT)","(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
331,(AMOUNTORIG),"(CSHYN_N, ACC_BUSINESSTYPE_CASA, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
333,(AMOUNT),"(CSHYN_N, AMOUNTORIG, ACC_BUSINESSTYPE_CASA)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0


In [87]:
filtered_rules.columns

Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'zhangs_metric'],
      dtype='object')

In [88]:
#  Focus on rules involving the ACC_BUSINESSTYPE_CASA, ACC_BUSINESSNO and AMOUNT#  Focus on rules involving the ACC_BUSINESSTYPE_CASA, ACC_BUSINESSNO and AMOUNT

# Check if 'antecedents' column exists before filtering
if 'antecedents' in filtered_rules.columns:
    casa_rules = filtered_rules[filtered_rules['antecedents'].apply(lambda x: 'ACC_BUSINESSTYPE_CASA' in x)]
else:
    print("Warning: 'antecedents' column not found in filtered_rules DataFrame.")
    casa_rules = filtered_rules  # Or handle the absence of the column as needed



In [89]:
casa_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
327,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
329,"(ACC_BUSINESSTYPE_CASA, AMOUNT)","(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
708,"(AMOUNTORIG, VALUEDATE, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
714,"(VALUEDATE, ACC_BUSINESSTYPE_CASA, AMOUNT)","(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
717,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, VALUEDATE, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
724,"(ACC_BUSINESSTYPE_CASA, AMOUNT)","(CSHYN_N, AMOUNTORIG, VALUEDATE)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
766,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
770,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, ACC_BUSINESSTYPE_CASA, AMOUNT)","(CSHYN_N, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
780,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
784,"(ACC_BUSINESSTYPE_CASA, AMOUNT)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0


In [91]:
casa_rules = casa_rules[casa_rules['consequents'].apply(lambda x: 'ACC_BUSINESSNO' in x or 'AMOUNT' in x)]


In [92]:
casa_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
327,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
708,"(AMOUNTORIG, VALUEDATE, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
717,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, VALUEDATE, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
766,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
780,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
1039,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, VALUEDATE, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
1053,"(ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, VALUEDATE, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
1064,"(AMOUNTORIG, VALUEDATE, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
1078,"(AMOUNTORIG, ACC_BUSINESSTYPE_CASA)","(CSHYN_N, ACC_BUSINESSNO_0553d33b8bbc8e3a944ff438a51a17cfb38fd0241667af98b6834685cc487ee6, VALUEDATE, AMOUNT)",0.991353,0.991111,0.991111,0.999756,1.008722,0.00857,36.442144,1.0
